In [1]:
# Import necessary packages
import pickle

import numpy as np
import pandas as pd
from evaluation_metrics import nse_loss
from hydrological_models import HBV
from modelcalibration_camelsus import ModelCalibrationCamelsUS as model_calibration


In [2]:
# Initialize information
path_data = "../../data/CAMELS_US"
path_additional_features = "../../data/CAMELS_US/pet_hargreaves.pickle"
path_output = "../results/HBV_extrapolation/"

input_variables = ["prcp(mm/day)", "pet(mm/day)", "tmax(C)", "tmin(C)"]
target_variables = ["QObs(mm/d)"]
forcing = "daymet"
warmup_period = 365

testing_period = "../../data/CAMELS_US/test_split_file_new.p"
hydrological_model = HBV()

In [3]:
# Read the calibration results by each method, and select the best case. In other words, select the calibrated
# parameters (for each basin) that gave best results.

# DREAM calibration
df_dream = pd.read_csv(path_output + hydrological_model.name + '_dream_summary.csv', dtype={'basin_id': str})
df_dream.set_index('basin_id', inplace=True)
basins_id = df_dream.index

# SCE calibration
df_sce = pd.read_csv(path_output + hydrological_model.name + '_sce_summary.csv', dtype={'basin_id': str})
df_sce.set_index('basin_id', inplace=True)

# The fist column of each dataset is the NSE in training.
nse_training = pd.concat([df_dream.iloc[:, 0], df_sce.iloc[:, 0]], axis=1,  keys=['dream', 'sce'])
max_value_index = nse_training.idxmax(axis=1)

# Select the best parameter set for each basin
parameter_sets = pd.concat([df_dream[max_value_index=='dream'].iloc[:, 1:-1], 
                            df_sce[max_value_index=='sce'].iloc[:, 1:-1]], axis=0)
parameter_sets= parameter_sets.reindex(df_dream.index)

Run the model for each basin, using the best calibration parameters

In [4]:
test_results = {}
NSE_testing =  []

# Loop that goes through each basin
for i, basin in enumerate(basins_id):
    testing_object = model_calibration(model = hydrological_model, 
                                       path_data = path_data,
                                       forcing=forcing,
                                       basin_id = basin, 
                                       input_variables = input_variables, 
                                       target_variables = target_variables,
                                       time_period = testing_period, 
                                       obj_func = None, 
                                       warmup_period = warmup_period,
                                       path_additional_features=path_additional_features)
    
    # Testing period ------------------------------------------
    q_sim = testing_object.simulation(parameter_sets.loc[basin].values)
    q_obs = testing_object.evaluation()

    # Calculate loss
    evaluation = q_obs[warmup_period:][testing_object.data_split[warmup_period:]]
    simulation = q_sim[warmup_period:][testing_object.data_split[warmup_period:]]

    # Store information in dataframe
    time_index = testing_object.timeseries['df'].index[warmup_period:][testing_object.data_split[warmup_period:]]
    df_discharge = pd.DataFrame(data={'y_obs': evaluation, 'y_sim': simulation}, index=time_index)
    test_results[basin] = df_discharge
    
    # Calculate NSE
    mask_nans = ~np.isnan(evaluation)
    NSE_testing.append(nse_loss(evaluation=evaluation[mask_nans].flatten(),
                                 simulation=simulation[mask_nans].flatten()))

# Save NSE values in csv
df_NSE = pd.DataFrame(data={'basin_id': basins_id,'NSE': NSE_testing})
df_NSE = df_NSE.set_index('basin_id')
df_NSE.to_csv(path_output+'/'+hydrological_model.name+'_NSE.csv', index=True, header=True)

# Save simulated values in pickle file
with open(path_output+"/HBV_results.pickle", "wb") as f:
    pickle.dump(test_results, f)